In [ ]:
## Run FIESTA for fieldwork location 

In [ ]:
###### Data

INSIVUMEH - Cahabón river flow measurement point 
Santa Cruz = Longitud 90° 26' 17.70" Latitud 15° 22' 19.49" (Coords =  15.37208, -90.43825) (insivumeh.gob.gt/wp-content/uploads/2023/02/BOLETIN-HIDROLOGICO-ESPECIAL-NO.-169-2023.pdf)
Sourced from = public information request to INSIVUMEH via email


###### Analysis Set up 

FIESTA - 

def Approach(Airflux, LWC, InterceptEff, Area):
    #NOTE: This function isn't actually used.
    #Approach to Fog Interception Model copied from FIESTA
    #Eq 1 = To model air flux one must model wind flows and speeds in complex topography. To 
    #       model LWC one must model the humidity of the air and the potential for 
    #       condensation of that humidity into liquid water. Knowing interception efficiency 
    #       means modelling the significant complexity of forest vegetation at multiple scales and 
    #       knowing the area exposed means analysing the spatial complexity of forest/pasture 
    #       mosaics when viewed from particular azimuthal wind directions and inclinations of 
    #       fog water flux.
    Fog_Interception = Air_Flux * LWC * InterceptEff * Area
    return Fog_Interception

In [ ]:
# Run FIESTA for Field Work location on 

FracTMCF_fw = 0#0.5
TreeFrac_fw = 0#0.5
FracClear_fw = 1 #0.5
FracPine_fw = 0
Elev_fw = 2070
CellSize_fw = 10000
Aspect_fw = 2
Slope_fw = 45
slope_rad = math.radians(Slope_fw)
Area3d_fw = CellSize_fw / math.cos(slope_rad)
Elev_ws = 1420
T_adj = 0.6 * ((Elev_fw - Elev_ws)/100)
T_max_fw = 17.1 - T_adj #Temperature from weather station adjusted for elevation
T_min_fw = 13.4 - T_adj
T_avg_fw = 15.2 - T_adj
RH_fw = 100 #From weather station 
Presh_fw = 1025 #hPa
Cld_fw = 1
Rad24_fw = 0
Rad6_fw = 1010
Rad12_fw = 245
Rad18_fw = 10
Month_fw = 1
LR_fw = 0
Wnd_d_fw = 15 #perpecdicular to slope
Wnd_spd_fw = 10 #from weather station
Prcp_fw = 0


Daily_deposition = np.zeros(4)
Daily_evap = np.zeros(4)
Daily_budget = np.zeros(4)
Daily_emergent_impaction = np.zeros(4)
for j in range(1, 5):
            #forestedgelenfacingm, emergentedgelenfacingm = Forest_Edge(TreeFrac[k,i], CellSize[k])
            forestedgelenfacingm_fw, emergentedgelenfacingm_fw = Forest_Edge(0.5, 10000) #1 hectare, half forested
            #emergentedgelenfacingm_fw = 100
            print('forestedgelenfacingm = ' + str(forestedgelenfacingm_fw)) 
            print('emergentedgelenfacingm = ' + str(emergentedgelenfacingm_fw)) 
            
            #DepositionFrac   = Sedimentation_Surface_Area(FracTMCF[k,i], FracClear[k,i], FracPine[k,i], LAI_TMCF, LAI_Clear, LAI_Pine, lss_TMCF, lss_Pine, lss_Clear)
            DepositionFrac_fw   = Sedimentation_Surface_Area(FracTMCF_fw, FracClear_fw, FracPine_fw, LAI_TMCF, LAI_Clear, LAI_Pine, lss_TMCF, lss_Pine, lss_Clear)
            print('DepositionFrac = ' + str(DepositionFrac_fw)) 
                      
            #EtFrac_total     = ET_LU(FracTMCF[k,i], FracClear[k,i], FracPine[k,i], LAI_TMCF, LAI_Clear, LAI_Pine)
            EtFrac_total_fw     = ET_LU(FracTMCF_fw, FracClear_fw , FracPine_fw, LAI_TMCF, LAI_Clear, LAI_Pine)
            print('EtFrac_total = ' + str(EtFrac_total_fw)) 
            
            #Tmp, AirRising   = Temps(j, T_max[i], T_min[i], T_avg[i], Elev[k]) #Tmp [C]
            Tmp_fw, AirRising_fw   = Temps(j, T_max_fw, T_min_fw, T_avg_fw, Elev_fw) #Tmp [C]
            AirRising_fw = 1
            print('Tmp = ' + str(Tmp_fw))
            print('AirRising = ' + str(AirRising_fw))
            
            #E                = Saturated_vapur_press(Tmp, RH[i]) #E [mb]
            E_fw                = Saturated_vapur_press(Tmp_fw, RH_fw) #E [mb]
            print('E = ' + str(E_fw))
            
            #AirDensity, AH   = Air_density(Presh[i], Tmp, E)
            AirDensity_fw, AH_fw   = Air_density(Presh_fw, Tmp_fw, E_fw)
            print('AirDensity = ' + str(AirDensity_fw))
            print('Absolute Humidity = ' + str(AH_fw)) 
                
            #LWC              = LiqWaterContent(AH, AH_max=0.02717, maxLWC=0.0002)
            LWC_fw              = LiqWaterContent(AH_fw, AH_max=0.02717, maxLWC=0.0002)
            LWC_fw = 0.00025
            print('LWC = ' + str(LWC_fw))
            
            #Td               = Dewpoint(E) # Td [C]
            Td_fw                = Dewpoint(E_fw) # Td [C]
            print('Td = ' + str(Td_fw))
            
            #LCL, fog         = LiftingCondensationLevel(Tmp, Td, Presh[i], Elev[k])
            LCL_fw, fog_fw         = LiftingCondensationLevel(Tmp_fw, Td_fw, Presh_fw, Elev_fw)
            print('LCL = ' + str(LCL_fw))
            print('fog = ' + str(fog_fw))
            
            
            if   j == 1:
                #SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad24[i])
                SolarR_ground_fw = EffectiveSolarRad(Cld_fw, fog_fw, Rad24_fw)
            elif j == 2:
                #SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad6[i])
                SolarR_ground_fw = EffectiveSolarRad(Cld_fw, fog_fw, Rad6_fw)
            elif j == 3:
                #SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad12[i])
                SolarR_ground_fw = EffectiveSolarRad(Cld_fw, fog_fw, Rad12_fw)
            elif j == 4:
                #SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad18[i]) 
                SolarR_ground_fw = EffectiveSolarRad(Cld_fw, fog_fw, Rad18_fw) 
            print('SloarR_ground = ' + str(SolarR_ground_fw))
            
            #NetMap            = NetRad(SolarR_ground, FracTMCF[k, i], FracPine[k, i], FracClear[k, i], Month[i], LR[i], SecondsInMonth) #NetMap [W/m2]
            NetMap_fw            = NetRad(SolarR_ground_fw, FracTMCF_fw, FracPine_fw, FracClear_fw, Month_fw, LR_fw, SecondsInMonth) #NetMap [W/m2]
            
            
            #Wind_spd_adj, Wind_dir_adj = WindSpeed(Wnd_d[i], Wnd_spd[i], Aspect[k])
            Wind_spd_adj_fw, Wind_dir_adj_fw = WindSpeed(Wnd_d_fw, Wnd_spd_fw, Aspect_fw)
            print('Wind_spd_adj = ' +str(Wind_spd_adj_fw))
            print('Wind_dir_adj = ' +str(Wind_dir_adj_fw))
            
            
            #Prec_corrected    = RainMod(Prcp[i], Wind_spd_adj, Wind_dir_adj, Slope[k], DropTermVel)
            Prec_corrected_fw    = RainMod(Prcp_fw, Wind_spd_adj_fw, Wind_dir_adj_fw, Slope_fw, DropTermVel)
            print('Prec_corrected = '  + str(Prec_corrected_fw))
            
            
            #EdgeImpactionFlux, EmergentImpactionFLux = EdgeImpactFlux(Wind_spd_adj, forestedgelenfacingm, emergentedgelenfacingm, LWC)
            EdgeImpactionFlux_fw, EmergentImpactionFLux_fw = EdgeImpactFlux(Wind_spd_adj_fw, forestedgelenfacingm_fw, emergentedgelenfacingm_fw, LWC_fw)
            print('EdgeImpactionFlux = '+ str(EdgeImpactionFlux_fw))
            print('EmergentImpactionFLux = ' + str(EmergentImpactionFLux_fw))
            
            
            #FogIntmm          = FogDeposition(Wind_spd_adj, TreeFrac[k, i], CellSize[k], LWC, FogSettlingVelocity, Area3d[k], EdgeImpactionFlux, EmergentImpactionFLux, AirRising, DepositionFrac, Cld[i], fog)
            FogIntmm_fw, DeposInterc          = FogDeposition(Wind_spd_adj_fw, TreeFrac_fw, CellSize_fw, LWC_fw, FogSettlingVelocity, Area3d_fw, EdgeImpactionFlux_fw, EmergentImpactionFLux_fw, AirRising_fw, DepositionFrac_fw, Cld_fw, fog_fw)
            print('FogIntmm = ' + str(FogIntmm_fw))
            print('Deposition_flux = ' + str(DeposInterc))
            
            #ActEvap           = Evap(Tmp, NetMap, EtFrac_total)
            ActEvap_fw           = Evap(Tmp_fw, NetMap_fw, EtFrac_total_fw)
            print('ActEvap = ' + str(ActEvap_fw))
            
            #Budget            = Budgets(Prec_corrected, FogIntmm, ActEvap)
            Budget_fw            = Budgets(Prec_corrected_fw, FogIntmm_fw, ActEvap_fw)
            print('Budget = ' + str(Budget_fw))
            
            print("     ")
            print("     ")
            print("------------")
            print("     ")
            print("     ")
            #Store Outputs 
            Daily_deposition[j-1] = FogIntmm_fw * 6 #6 hours per timestep
            Daily_evap[j-1] = ActEvap_fw
            Daily_budget[j-1] = Budget_fw
            #fog_val[k, i, j-1] = fog_fw
            #LCL_val[k, i, j-1] = LCL_fw
            #Daily_emergent_impaction[j-1] =  
            
            
            
np.sum(Daily_deposition[j-1])